# Task 1 Machine learning on tabular mushrooms

# Task 2 Sentiment analysis 

# Task 3 Convolutional neural networks

In this task, the assignment is about training a convolutional neural network (CNN) as a binary classifier from the dataset that we have been provided with. This is a CIFAR-10 dataset that consists of 60000 images that are 32x32 colored images and will identify one of the following categories; airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck. Each of these has 6000 images that are divided into 50000 for the training model and 10000 for the testing model.

In the following code block, we are importing the necessary libraries and modules to build and train the convolutional neural network (CNN) for binary image classification using the Dataset of CIFAR-10.

While researching for this assignment we found a couple of different pre-trained CNN models we could use for this project. Some of the choices were ResNet-50/101/152, Inception-v3/v4, MobileNetV1/V2/V3. We decided not to use these because of the computing power that would be unsatisfactory for some systems with the amount of layers that the models have. We landed upon the VGG16 model that is a deep CNN model that has 16 layers. It is a less complex model compared to the previous ones we have mentioned, but it is more complex then a 3-5 layer CNN. We chose VGG16 at first because we ran into some errors while trying to create our own 3-5 layer CNN using a pre-trained CNN model that has strong performance in particular image classification.

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt

2023-05-02 11:15:26.677298: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-02 11:15:26.714365: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-02 11:15:26.911612: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-02 11:15:26.912398: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 11:15:27.849597: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

The code block defines the unpickle function to load CIFAR-10 dataset files, which are serialized using Python's pickle module. The function takes a file path as input, reads the dataset file, and returns a dictionary with the data.

In [2]:
# Define the function to load our dataset files
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

Loading and Preprocessing CIFAR-10 Dataset

The `load_cifar10_data` function loads and preprocesses the CIFAR-10 dataset so that we can apply it to our CNN later. It takes the path to the data directory and returns preprocessed training and testing data with their labels.

In [3]:
# Load and preprocess the dataset
def load_cifar10_data(data_dir):
    train_data = None
    train_labels = []

    for i in range(1, 6):
        data_dict = unpickle(os.path.join(data_dir, f'data_batch_{i}'))
        if i == 1:
            train_data = data_dict[b'data']
        else:
            train_data = np.vstack((train_data, data_dict[b'data']))
        train_labels += data_dict[b'labels']

    test_data_dict = unpickle(os.path.join(data_dir, 'test_batch'))
    test_data = test_data_dict[b'data']
    test_labels = test_data_dict[b'labels']

    train_data = train_data.reshape((len(train_data), 3, 32, 32))
    train_data = np.rollaxis(train_data, 1, 4)
    train_labels = np.array(train_labels)

    test_data = test_data.reshape((len(test_data), 3, 32, 32))
    test_data = np.rollaxis(test_data, 1, 4)
    test_labels = np.array(test_labels)

    return train_data, train_labels, test_data, test_labels

Loading CIFAR-10 Dataset into Variables

We use load_cifar10_data to load and preprocess the CIFAR-10 dataset. These arrays are stored in corresponding variables for later use in the neural network model training process.

In [4]:
data_dir = './cifar-10-batches-py'
x_train, y_train, x_test, y_test = load_cifar10_data(data_dir)

Preprocessing and Normalizing Input Data

In this step, we preprocess input data by:

1. We divide and the pixel values by 225 and normalizing them to the range [0, 1]. This normalization helps improve the training process by ensuring that the input values are within a similar scale.
2. Choosing a category for binary classification.
3. Converting class labels to one-hot encoding using tf.keras.utils.to_categorical.

This prepares the dataset for training the CNN model by ensuring input values are within a similar scale and using one-hot encoding for our 2 categories that we have later of ["not airplane", "airplane"].

By normalizing the input data and converting the class labels to one-hot encoding, we prepare our dataset for training the CNN model.

In [5]:
# Preprocess input data, and normalize the input data 
x_train = x_train / 255.0
x_test = x_test / 255.0

chosen_category = 0

# Convert class labels to one-hot encoding
y_train = np.where(y_train == chosen_category, 1, 0)
y_test = np.where(y_test == chosen_category, 1, 0)

Plotting Validation Loss

The function plot_validation_loss visualizes the validation loss during training. It takes the training history object from the model's fit method and generates a plot of validation loss over epochs.

We are going to use this function to analyze the training process and identify any issues, such as overfitting or underfitting, based on the validation loss curve.

In [6]:
# Function to plot validation loss
def plot_validation_loss(history):
    plt.plot(history.history['val_loss'])
    plt.title('Validation Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Validation'], loc='upper right')
    plt.show()

Plotting Model Accuracy

The `plot_accuracy(history)` function plots training and validation accuracy over epochs. It takes the training history object from the model's `fit` method.

The plot helps further observe the model's performance and identify potential gaps between training and validation accuracy curves.

In [7]:
def plot_accuracy(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

Loading Pre-trained VGG16 Model

The code block loads the pre-trained VGG16 model using `tensorflow.keras.applications.VGG16` as the base for a binary image classifier.

Arguments for VGG16 function:
1. `weights`: 'imagenet' weights, trained on ImageNet dataset.
2. `include_top`: Set to False to exclude original classification layer, as a custom binary classification layer will be added.
3. `input_shape`: Tuple with input shape (32, 32, 3) for CIFAR-10 images. 32x32 pixels size with 3 color channels

The loaded VGG16 model is stored in the `base_model` variable to build the custom binary classifier.


In [8]:
# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

2023-05-02 11:15:30.378314: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Modifying VGG16 Model for Binary Classification

Here we modify the pre-trained VGG16 model to create a binary image classifier:

1. Add a GlobalAveragePooling2D layer to reduce spatial dimensions of feature maps, preventing our model to be overfitted.
2. Add a Dense layer with 1024 units and ReLU activation to learn higher-level features.
3. Change the output layer to a single unit with sigmoid activation for binary classification. The sigmoid function outputs a probability for the input image belonging to the chosen category which is [0, 1].

In [9]:
# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
x = Dense(1024, activation='relu')(x)

# Add the final output layer for 10-class classification
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

Fine-tuning Custom Binary Classifier Model

Here we fine-tune the binary classifier on the dataset:

1. Freeze Convolutional Layers: Set the trainable attribute of the VGG16 base model layers to False, so only the added fully connected layers are trained, speeding up the process.
2. Compile the Model: Use the Adam optimizer with a 0.0001 learning rate, 'binary_crossentropy' loss, and 'accuracy' as the evaluation metric. We are using 0.0001 learning rate to Preserve pre-trained features that ensures we don't make drastic updates that could degrade the model's performance, more stable convergence, preventing overshooting optimal weights and learning slowly helps the model generalize better, reducing overfitting.
3. Train the Model: Train using the preprocessed CIFAR-10 dataset with x_train, y_train, and 5 epochs. Monitor performance using x_test and y_test as validation data.

The training history is stored for plotting accuracy and loss over time, helping identify model performance and overfitting.

In [10]:
# Fine-tune only the top layers (freeze all convolutional layers)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))

Epoch 1/3
  95/1563 [>.............................] - ETA: 2:25 - loss: 0.3150 - accuracy: 0.8924

KeyboardInterrupt: 

Plotting Validation Loss

Using the `plot_validation_loss(history)` function, we plot the validation loss over training epochs. The history variable holds training information, including validation loss per epoch.

By analyzing the plot, we can assess how well the model generalizes to unseen data.

Despite using 5 to 25 epochs, the model did not converge but still provided correct binary classification.

In [ ]:
# Plotting the validation loss
plot_validation_loss(history)

Plotting Model Accuracy

We use the `plot_accuracy(history)` function to visualize training and validation accuracy over training epochs. The history variable holds training information, including accuracy per epoch.

The function plots the training and validation accuracy, displaying the model's performance. By examining the plot, we can assess the model's performance and identify further issues based on the gap between the curves. A well-performing model should have a smaller gap and high accuracy on both sets, indicating effective learning and good generalization to unseen data.

In [ ]:
# Plotting the accuracy
plot_accuracy(history)

Loading and Preprocessing a New Image

To load and preprocess a new image for classification:

1. Load New Image: The image needs to be resized so that it matches the model's input size (32 x 32 pixels).
2. Normalize the Image: Divide the image's pixel values by 255.0 to normalize it, ensuring pixel values are in the same 0 to 1 range as the training data.

The new image is now ready for classification by the model. We have tested that the images are the images are the same format. We prefered using pngs.

In [ ]:
# Load a new image and preprocess it
new_image_path = './new_images/picture_1.png'# Here is the pathing for the image so if you want to add an image just past it into this folder and change the name.
new_image = image.load_img(new_image_path, target_size=(32, 32))
new_image = image.img_to_array(new_image)
new_image = np.expand_dims(new_image, axis=0)
new_image = new_image / 255.0


Predicting the Class of a New Image

To predict the class of a preprocessed new image using the trained model:

1. Predict the Class: Use `model.predict()` to predict the class, which returns the predicted probability for the positive class (chosen category).

2. Determine the Predicted Class: Set a 0.5 threshold for deciding the predicted class. If the predicted probability is above 0.5, assign the positive class (1); otherwise, assign the negative class (0).

`predicted_class` now contains the predicted class of the new image, indicating if the model classified the image as part of the chosen category or not.

In [ ]:
# Predict the class of the new image
prediction = model.predict(new_image)
predicted_class = 1 if prediction[0][0] > 0.5 else 0


Displaying the Predicted Class

We categories our binary values of ['not airplane', 'airplane']

This prints outs an output, showing if the model classified the new image as part of the chosen category (airplane) or not.


In [ ]:
categories = ['not airplane', 'airplane']
print(f"The image is predicted to be {categories[predicted_class]}.")